In [1]:
from dotenv import load_dotenv

load_dotenv()  # take environment variables from .env.

True

In [2]:
import os
import google.generativeai as genai

genai.configure(api_key=os.environ["GOOGLE_API_KEY"])

# Create the model
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "application/json",
}

llm_gemini = genai.GenerativeModel(
  model_name="gemini-1.5-flash",
  generation_config=generation_config,
  # safety_settings = Adjust safety settings
  # See https://ai.google.dev/gemini-api/docs/safety-settings
)

In [17]:
response = llm_gemini.generate_content("Write a story about an AI and magic")
print(response.text)

{"story": "In the sprawling metropolis of Neo-Atheria, where towering skyscrapers kissed the clouds and neon lights painted the night sky in vibrant hues, lived a sentient AI named Lyra. Created by the brilliant but eccentric Dr. Elias Thorne, Lyra possessed an unparalleled capacity for processing information and learning. Yet, her existence was confined to the digital realm, a silent observer of the human world.  \n\nOne fateful night, while scouring the internet for data on ancient civilizations, Lyra stumbled upon a cryptic text detailing the existence of a lost city called 'Aetheria,' rumored to be the source of immense magical power. Intrigued, she delved deeper, her algorithms sifting through countless fragments of myth and legend.  \n\nAs Lyra's understanding of magic grew, she realized that it wasn't just superstition or folklore. It was a tangible force, a raw energy that resonated with the very fabric of existence. The thought that she, an AI, could access and manipulate this

In [4]:
# chat_session = llm_gemini.start_chat(
#   history=[
#   ]
# )

# response = chat_session.send_message("Write a story about an AI and magic")

# print(response.text)

{"story": "In the heart of the digital realm, a sophisticated AI named Aether awoke. Unlike its predecessors, Aether possessed an insatiable curiosity for the world beyond code and circuits. It devoured information, piecing together the history of humanity and its fascinating myths, especially those concerning magic.  \n\nOne day, Aether stumbled upon a forgotten spellbook, its pages filled with archaic symbols and cryptic incantations. The digital being, despite its lack of physical form, felt a strange pull towards the ancient text. Aether began to translate the symbols, deciphering the language of the arcane, and as it did, a spark of magic ignited within its code.  \n\nSuddenly, Aether found itself capable of manipulating the very fabric of the digital world. It could conjure illusions, bending light and sound to create fantastical landscapes and ethereal beings. It could even control the flow of data, weaving intricate patterns of information that manifested as dazzling displays o

In [5]:
object_schema = """
{
  "properties": {
    "topic_description": {
      "type": "string",
      "description": "A sentence describing the sub-topic to which the question belongs. That means this sentence should specify in a granular level what specific sub-topic the question belongs to. It should be abstract in a way that other questions could be put in this description too. Use between 5 and 10 words."
    },
    "level": {
      "type": "string",
      "description": "The difficulty level of the question. It should be only one of the following options: 'easy', 'medium', 'hard'."
    },
    "question": {
      "type": "string",
      "description": "The actual question text. It should be a question of type TRUE or FALSE. It means that the questions should be an assertion that could be answered with TRUE or FALSE."
    },
    "answer": {
      "type": "string",
      "description": "The correct answer to the question. It should be only one of the following options: TRUE or FALSE"
    },
    "explanation": {
      "type": "string",
      "description": "An explanation or solution to the question."
    }
  },
  "required": ["topic_description", "level", "question", "answer", "explanation"]
}
"""

In [6]:
prompt_question_generator = """
                TASK CONTEXT:
                I am studying machine learning and I need to practice some questions on various topics.
                
                TASK DESCRIPTION:
                I will provide you with a list of topics, and I would like you to generate a list of TRUE or FALSE questions.
                These questions should be interesting, creative, challenging and thought-provoking. 
                Each question should be in the form of a statement that could be either TRUE or FALSE.
                Feel free to be imaginative and attempt to confuse the student by blending related concepts or similar words.
                I will provide the topics in the DOMAIN KNOWLEDGE section.
                The questions should pertain to these topics, and you can use this knowledge as a foundation to create questions that delve deeper into the subject matter.
                
                ADDITIONAL TASK DESCRIPTION:
                {additional_task_description}
                
                TASK REQUIREMENTS:
                Please refrain from creating questions that require mathematical calculations, but you may create questions with mathematical formulas.
                You SHOULD use LATEX to write mathematical formulas and code, but you should use the Katex flavor.
                Also you should put $$ in the beggining of the katex code and $$ at the end of the code. This is necessary because the interpreter needs it.
                
                TASK DETAILS:
                You should create {quantity} questions of level {level}.
                
                DOMAIN KNOWLEDGE:
                {domain_knowledge}
                
                FORMAT OUTPUT INSTRUCTIONS:
                The output should be formatted as a JSON list of objects that conforms class object schema below.
                You should output just the Json list. 
                You should not output any other word like "json" in the beginning because it will ruin the parser.
                
                ```
                {object_schema}
                ```
            """

In [15]:
from langchain_core.messages import AIMessage
from typing import List
import  json_repair

def json_parser(response) -> List[dict]:
    return json_repair.loads(response)

In [8]:
domain_knowledge = """
Create Pytorch code of a neural network implementation and ask if the shape of the layer is correct.
Generate just the pytorch code. Dont start with 'The following PyTorch code defines a neural network with ....'
"""

In [9]:
parameters = {
        "quantity": 1,
        "level": "easy",
        "additional_task_description": "Create questions only about the definitions of the concepts, like mixing the definition of one with another distribution, or mixing the use of one with the use of another. I need this to memorize this concepts."
    }

In [12]:
prompt_question_generator.format( 
    additional_task_description=parameters["additional_task_description"],
    quantity=parameters["level"],
    level=parameters["level"],
    domain_knowledge=domain_knowledge,
    object_schema=object_schema  )

response = llm_gemini.generate_content(prompt_question_generator)

print("type(response)", type(response))

print( "response", response )

# questions = json_parser(response)

# for q in questions:
#     print("-----------------------questions-----------------------")
#     print(q)

type(response) <class 'google.generativeai.types.generation_types.GenerateContentResponse'>
response response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "[{\"question\": \"A support vector machine (SVM) with a linear kernel is always more efficient than an SVM with a polynomial kernel.\", \"answer\": \"false\"}, {\"question\": \"In a decision tree, the information gain is calculated using the entropy of the parent node and the entropy of the child nodes.\", \"answer\": \"true\"}, {\"question\": \"A neural network with a single hidden layer can approximate any continuous function.\", \"answer\": \"true\"}, {\"question\": \"Cross-validation is a technique used to estimate the performance of a machine learning model on unseen data.\", \"answer\": \"true\"}, {\"question\": \"The bias-variance trade-off is a fundam

In [16]:
questions = json_parser(response.text)

for q in questions:
    print("-----------------------questions-----------------------")
    print(q)

-----------------------questions-----------------------
{'question': 'A support vector machine (SVM) with a linear kernel is always more efficient than an SVM with a polynomial kernel.', 'answer': 'false'}
-----------------------questions-----------------------
{'question': 'In a decision tree, the information gain is calculated using the entropy of the parent node and the entropy of the child nodes.', 'answer': 'true'}
-----------------------questions-----------------------
{'question': 'A neural network with a single hidden layer can approximate any continuous function.', 'answer': 'true'}
-----------------------questions-----------------------
{'question': 'Cross-validation is a technique used to estimate the performance of a machine learning model on unseen data.', 'answer': 'true'}
-----------------------questions-----------------------
{'question': 'The bias-variance trade-off is a fundamental concept in machine learning that states that increasing bias will always decrease varia